# DFW Weather 

This notebook scrapes the weather data from wunderground and saves it to the database.  Here's an example page:

https://www.wunderground.com/history/airportfrompws/KDFW/2015/1/1/CustomHistory.html?dayend=15&monthend=8&yearend=2019&req_city=&req_state=&req_statename=&reqdb.zip=&reqdb.magic=&reqdb.wmo=

In [23]:
import os
import requests
import pandas as pd

def get_html_from_wunderground(year):

    url = "https://www.wunderground.com/history/airportfrompws/KDFW/"
    url += "{}/1/1/CustomHistory.html?".format(year)
    url += "dayend=31&monthend=12&yearend={}".format(year)
    
    page = requests.get(url)
    assert page.status_code == 200
    return page.text

def get_df_from_html(year, html):
    res = pd.read_html(html)
    df = res[1].copy()
    columns = ['day', 
               'high_temp', 'mean_temp', 'low_temp', 
               'high_dew', 'mean_dew', 'low_dew',
               'high_hum', 'mean_hum', 'low_hum',
               'high_pres', 'mean_pres', 'low_pres',
               'high_vis', 'mean_vis', 'low_vis',
               'high_wind', 'mean_wind', 'low_wind',
               'precip', 'events']

    df.columns = columns
    df['month'] = None
    df['year'] = year
    df = df[ ['year', 'month'] + columns ].copy()
    return df

def clean_df(df):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    rows = []
    current_month = None
    for row in df.to_dict(orient='records'):
        if row['day'] in months:
            current_month = row['day']
        else:
            row['month'] = current_month
            rows.append(row)

    print(len(rows))
    return pd.DataFrame(rows)

In [24]:
year = 2015

dfs = []
for year in ['2015', '2016', '2017', '2018', '2019']:

    html = get_html_from_wunderground(year)
    df = get_df_from_html(year, html)
    clean = clean_df(df)
    print(year, clean.shape)
    clean.to_csv("weather_{}.csv".format(year), index=False)
    dfs.append(clean)

365
2015 (365, 23)
366
2016 (366, 23)
365
2017 (365, 23)
365
2018 (365, 23)
190
2019 (190, 23)


Create one dataframe

In [25]:
df = pd.concat(dfs)

In [34]:
df['period'] = df['year'] + "-" + df['month'] + "-" + df['day']
df['period'] = pd.to_datetime(df['period'])

Convert numeric columns

In [67]:
for col in df.columns:
    if col not in ['events', 'period', 'month']:
        try:
            df[col] = df[col].map(lambda x: x if x != '-' else 0 )
            df[col] = df[col].map(lambda x: x if x != 'T' else 0 )
            df[col] = df[col].astype('float')
        except ValueError as e:
            print(e)

Map events

In [82]:
event_list = clean['events'].fillna("").map(lambda x: x.replace("\t,", ":")).drop_duplicates().values

In [83]:
from operator import add
from functools import reduce

event_list = reduce(add, [ x.split(":") for x in event_list ])

In [89]:
event_list = list(set([ x.strip() for x in event_list ]))
event_list = [ x for x in event_list if len(x)>0]
event_list

['Thunderstorm', 'Fog', 'Snow', 'Hail', 'Rain']

In [90]:
for event in event_list:
    df[event.lower()] = df['events'].map(lambda x: 1 if str(x).find(event)>=0 else 0)

In [93]:
df[[ x.lower() for x in event_list]].head()

,thunderstorm,fog,snow,hail,rain
0,0,0,0,0,1
1,1,0,0,0,1
2,1,0,0,0,1
3,0,0,0,0,0
4,0,0,0,0,0


Save to database

In [96]:
import athena
db = athena.database.AthenDatabase(cache=True, write=True)

In [97]:
df.to_sql("weather_table", db.engine, if_exists='replace', index=False, method='multi')
db.save_as(df, "weather_table.csv")

Cleanup

In [98]:
!rm weather_20*.csv